In [1]:
import sqlite3
import utils
import pandas as pd
from sklearn.model_selection import train_test_split
from numba import cuda

from sklearn.metrics import f1_score
from sklearn.dummy import DummyClassifier

from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

import numpy as np
import torch

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

from keras.layers import Conv1D, GlobalMaxPooling1D,LSTM

C:\Users\X\AppData\Local\Continuum\anaconda3\envs\mlgpu\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
import keras.backend as K

#dtype='float16'
#K.set_floatx(dtype)

# default is 1e-7 which is too small for float16.  Without adjusting the epsilon, we will get NaN predictions because of divide by zero problems
#K.set_epsilon(1e-3) 

In [3]:
import tensorflow as tf

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6796095877393987637
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4557766656
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18380116288530818078
physical_device_desc: "device: 0, name: GeForce RTX 2060, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


In [5]:
def free_gpu_mem():
    cuda.select_device(0)
    cuda.close()

In [6]:
def limit_mem():
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))

In [7]:
word_vec_size=300

In [8]:
def w2v_per_token_wiki(word_vec_size):
    conn=sqlite3.connect('songs.db')
    cur=conn.cursor()
    example_text=cur.execute("select Lyrics from song where Lyrics is not null").fetchall()
    labels=[i[0] for i in cur.execute("select Economic from song where Lyrics is not null").fetchall()]
    songs=[]
    for i in example_text:
        songs.append(utils.tokenize_word(utils.clean_text(i[0])))

    embeddings_index,songs=utils.get_embedding_dict(songs,'fasttext-wiki-news-subwords-300')
    print(len(embeddings_index.keys()))
    max_words=len(embeddings_index.keys())
    X,y=np.array(songs),np.array(labels)

    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(X)
    sequences = tokenizer.texts_to_sequences(X)

    length = []
    for x in X:
        length.append(len(x))

    X = pad_sequences(sequences, maxlen=max(length)+2)

    embedding_matrix = np.zeros((max_words, word_vec_size))
    for word, i in tokenizer.word_index.items():
        if i >= max_words:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.5)
    
    
    return x_train,x_test,y_train,y_test,embedding_matrix,max(length)+2,max_words

In [9]:
x_train,x_test,y_train,y_test,embedding_matrix,max_sent_len,max_words=w2v_per_token_wiki(word_vec_size)

19205


In [10]:
def dummy(x_train,x_test,y_train,y_test):
    dummy = DummyClassifier()
    dummy.fit(x_train, y_train)
    y_pred=dummy.predict(x_test)
    return f1_score(y_test,y_pred)

In [11]:
print(dummy(x_train,x_test,y_train,y_test))

0.43991853360488803


In [12]:
def dnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model_ptw2v = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=False)
    model_ptw2v.add(e)
    model_ptw2v.add(Flatten())
    model_ptw2v.add(Dense(128, activation='relu'))
    model_ptw2v.add(Dropout(0.25))
    model_ptw2v.add(Dense(1, activation='sigmoid'))
    model_ptw2v.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_ptw2v.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=64, verbose=2)
    y_pred = [int(round(i[0])) for i in model_ptw2v.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [13]:
print(dnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 9s - loss: 0.6844 - acc: 0.5888 - val_loss: 0.6078 - val_acc: 0.6352
Epoch 2/100
 - 5s - loss: 0.4139 - acc: 0.8139 - val_loss: 0.6099 - val_acc: 0.6436
Epoch 3/100
 - 5s - loss: 0.2539 - acc: 0.9243 - val_loss: 0.6555 - val_acc: 0.6556
Epoch 4/100
 - 5s - loss: 0.1671 - acc: 0.9621 - val_loss: 0.6527 - val_acc: 0.6469
Epoch 5/100
 - 5s - loss: 0.1039 - acc: 0.9769 - val_loss: 0.7054 - val_acc: 0.6535
Epoch 6/100
 - 5s - loss: 0.0781 - acc: 0.9814 - val_loss: 0.7557 - val_acc: 0.6593
Epoch 7/100
 - 5s - loss: 0.0607 - acc: 0.9853 - val_loss: 0.8174 - val_acc: 0.6562
Epoch 8/100
 - 5s - loss: 0.0534 - acc: 0.9871 - val_loss: 0.7982 - val_acc: 0.6572
Epoch 9/100
 - 5s - loss: 0.0576 - acc: 0.985

Epoch 86/100
 - 5s - loss: 0.0174 - acc: 0.9901 - val_loss: 1.6958 - val_acc: 0.6559
Epoch 87/100
 - 5s - loss: 0.0197 - acc: 0.9907 - val_loss: 1.6755 - val_acc: 0.6529
Epoch 88/100
 - 5s - loss: 0.0253 - acc: 0.9901 - val_loss: 1.6089 - val_acc: 0.6572
Epoch 89/100
 - 5s - loss: 0.0217 - acc: 0.9898 - val_loss: 1.6705 - val_acc: 0.6562
Epoch 90/100
 - 5s - loss: 0.0211 - acc: 0.9895 - val_loss: 1.6979 - val_acc: 0.6575
Epoch 91/100
 - 5s - loss: 0.0220 - acc: 0.9898 - val_loss: 1.6899 - val_acc: 0.6562
Epoch 92/100
 - 5s - loss: 0.0208 - acc: 0.9907 - val_loss: 1.7346 - val_acc: 0.6587
Epoch 93/100
 - 5s - loss: 0.0218 - acc: 0.9892 - val_loss: 1.7056 - val_acc: 0.6529
Epoch 94/100
 - 5s - loss: 0.0203 - acc: 0.9916 - val_loss: 1.6889 - val_acc: 0.6593
Epoch 95/100
 - 5s - loss: 0.0154 - acc: 0.9919 - val_loss: 1.7940 - val_acc: 0.6569
Epoch 96/100
 - 5s - loss: 0.0210 - acc: 0.9910 - val_loss: 1.7595 - val_acc: 0.6614
Epoch 97/100
 - 5s - loss: 0.0155 - acc: 0.9925 - val_loss: 1.780

In [14]:
def dnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model_ptw2v = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=True)
    model_ptw2v.add(e)
    model_ptw2v.add(Flatten())
    model_ptw2v.add(Dense(128, activation='relu'))
    model_ptw2v.add(Dropout(0.25))
    model_ptw2v.add(Dense(1, activation='sigmoid'))
    model_ptw2v.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model_ptw2v.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=64, verbose=2)
    y_pred = [int(round(i[0])) for i in model_ptw2v.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [15]:
print(dnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 10s - loss: 0.6856 - acc: 0.5741 - val_loss: 0.6248 - val_acc: 0.6343
Epoch 2/100
 - 5s - loss: 0.4940 - acc: 0.7650 - val_loss: 0.5553 - val_acc: 0.6499
Epoch 3/100
 - 6s - loss: 0.2116 - acc: 0.9360 - val_loss: 0.6391 - val_acc: 0.6578
Epoch 4/100
 - 5s - loss: 0.0799 - acc: 0.9796 - val_loss: 0.6790 - val_acc: 0.6566
Epoch 5/100
 - 5s - loss: 0.0653 - acc: 0.9838 - val_loss: 0.7432 - val_acc: 0.6532
Epoch 6/100
 - 5s - loss: 0.0500 - acc: 0.9841 - val_loss: 0.7936 - val_acc: 0.6526
Epoch 7/100
 - 5s - loss: 0.0480 - acc: 0.9856 - val_loss: 0.8108 - val_acc: 0.6611
Epoch 8/100
 - 5s - loss: 0.0405 - acc: 0.9862 - val_loss: 0.8930 - val_acc: 0.6544
Epoch 9/100
 - 5s - loss: 0.0373 - acc: 0.9877 - val_loss: 1.0196 - val_acc: 0.6547
Epoch 10/100
 - 5s - loss: 0.0448 - acc: 0.9856 - val_loss: 0.9959 - val_acc: 0.6547
Epoch 11/100
 - 5s - loss: 0.0360 - acc: 0.9880 - val_loss: 0.9365 - val_acc: 0.6584
Epoch 12/100
 - 6s - loss

Epoch 97/100
 - 5s - loss: 0.0210 - acc: 0.9910 - val_loss: 2.2826 - val_acc: 0.6593
Epoch 98/100
 - 5s - loss: 0.0202 - acc: 0.9898 - val_loss: 2.0577 - val_acc: 0.6547
Epoch 99/100
 - 5s - loss: 0.0214 - acc: 0.9910 - val_loss: 2.0613 - val_acc: 0.6550
Epoch 100/100
 - 5s - loss: 0.0200 - acc: 0.9904 - val_loss: 2.1573 - val_acc: 0.6572
0.6267582597317631


In [16]:
def cnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=False)
    model.add(e)
    model.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=16, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [17]:
print(cnn_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 8s - loss: 0.6837 - acc: 0.5633 - val_loss: 0.6732 - val_acc: 0.5622
Epoch 2/100
 - 4s - loss: 0.6430 - acc: 0.6255 - val_loss: 0.6458 - val_acc: 0.6139
Epoch 3/100
 - 4s - loss: 0.5495 - acc: 0.7054 - val_loss: 0.6912 - val_acc: 0.5820
Epoch 4/100
 - 5s - loss: 0.4370 - acc: 0.7980 - val_loss: 0.6279 - val_acc: 0.6605
Epoch 5/100
 - 5s - loss: 0.3113 - acc: 0.8717 - val_loss: 0.6680 - val_acc: 0.6599
Epoch 6/100
 - 5s - loss: 0.2012 - acc: 0.9303 - val_loss: 0.7658 - val_acc: 0.6415
Epoch 7/100
 - 4s - loss: 0.1482 - acc: 0.9579 - val_loss: 0.7935 - val_acc: 0.6638
Epoch 8/100
 - 4s - loss: 0.1155 - acc: 0.9675 - val_loss: 0.8463 - val_acc: 0.6484
Epoch 9/100
 - 4s - loss: 0.0826 - acc: 0.9823 - val_loss: 1.0409 - val_acc: 0.6442
Epoch 10/100
 - 4s - loss: 0.0880 - acc: 0.9775 - val_loss: 1.0063 - val_acc: 0.6614
Epoch 11/100
 - 4s - loss: 0.0679

Epoch 94/100
 - 3s - loss: 0.0251 - acc: 0.9880 - val_loss: 2.4147 - val_acc: 0.6445
Epoch 95/100
 - 3s - loss: 0.0197 - acc: 0.9895 - val_loss: 2.6619 - val_acc: 0.6451
Epoch 96/100
 - 3s - loss: 0.0179 - acc: 0.9898 - val_loss: 2.7299 - val_acc: 0.6463
Epoch 97/100
 - 3s - loss: 0.0167 - acc: 0.9907 - val_loss: 2.7899 - val_acc: 0.6502
Epoch 98/100
 - 3s - loss: 0.0167 - acc: 0.9910 - val_loss: 2.8977 - val_acc: 0.6520
Epoch 99/100
 - 3s - loss: 0.0165 - acc: 0.9910 - val_loss: 3.0601 - val_acc: 0.6556
Epoch 100/100
 - 3s - loss: 0.0178 - acc: 0.9904 - val_loss: 2.9184 - val_acc: 0.6538
0.5678919729932483


In [18]:
def cnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=True)
    model.add(e)
    model.add(Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=100, batch_size=16, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [19]:
print(cnn_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/100
 - 5s - loss: 0.6614 - acc: 0.5876 - val_loss: 0.6020 - val_acc: 0.6508
Epoch 2/100
 - 4s - loss: 0.4301 - acc: 0.7992 - val_loss: 0.6123 - val_acc: 0.6538
Epoch 3/100
 - 4s - loss: 0.1509 - acc: 0.9597 - val_loss: 0.7207 - val_acc: 0.6608
Epoch 4/100
 - 4s - loss: 0.0821 - acc: 0.9826 - val_loss: 0.7908 - val_acc: 0.6611
Epoch 5/100
 - 4s - loss: 0.0652 - acc: 0.9859 - val_loss: 0.7837 - val_acc: 0.6581
Epoch 6/100
 - 4s - loss: 0.0582 - acc: 0.9859 - val_loss: 0.8936 - val_acc: 0.6529
Epoch 7/100
 - 4s - loss: 0.0558 - acc: 0.9853 - val_loss: 0.7972 - val_acc: 0.6499
Epoch 8/100
 - 4s - loss: 0.0414 - acc: 0.9871 - val_loss: 0.8575 - val_acc: 0.6409
Epoch 9/100
 - 4s - loss: 0.0329 - acc: 0.9868 - val_loss: 0.9174 - val_acc: 0.6421
Epoch 10/100
 - 4s - loss: 0.0290 - acc: 0.9871 - val_loss: 0.9419 - val_acc: 0.6451
Epoch 11/100
 - 4s - loss: 0.0256 - acc: 0.9868 - val_loss: 0.9267 - val_acc: 0.6421
Epoch 12/100
 - 4s - loss:

Epoch 97/100
 - 4s - loss: 0.0206 - acc: 0.9889 - val_loss: 2.5902 - val_acc: 0.6556
Epoch 98/100
 - 4s - loss: 0.0177 - acc: 0.9904 - val_loss: 2.6841 - val_acc: 0.6553
Epoch 99/100
 - 4s - loss: 0.0169 - acc: 0.9907 - val_loss: 2.7524 - val_acc: 0.6566
Epoch 100/100
 - 4s - loss: 0.0168 - acc: 0.9901 - val_loss: 2.7981 - val_acc: 0.6562
0.6161073825503355


In [20]:
def lstm_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=False)
    model.add(e)
    model.add(LSTM(100))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [21]:
print(lstm_pretrained(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/10
 - 803s - loss: 0.6842 - acc: 0.5627 - val_loss: 0.6852 - val_acc: 0.5580
Epoch 2/10
 - 896s - loss: 0.6772 - acc: 0.5720 - val_loss: 0.6820 - val_acc: 0.5607
Epoch 3/10
 - 884s - loss: 0.6666 - acc: 0.5918 - val_loss: 0.6787 - val_acc: 0.5676
Epoch 4/10
 - 809s - loss: 0.6459 - acc: 0.6354 - val_loss: 0.6691 - val_acc: 0.5950
Epoch 5/10
 - 826s - loss: 0.6211 - acc: 0.6592 - val_loss: 0.6743 - val_acc: 0.5986
Epoch 6/10
 - 748s - loss: 0.5911 - acc: 0.6874 - val_loss: 0.6729 - val_acc: 0.5995
Epoch 7/10
 - 739s - loss: 0.5684 - acc: 0.7127 - val_loss: 0.7002 - val_acc: 0.6079
Epoch 8/10
 - 732s - loss: 0.5332 - acc: 0.7349 - val_loss: 0.7073 - val_acc: 0.6172
Epoch 9/10
 - 732s - loss: 0.5028 - acc: 0.7487 - val_loss: 0.6977 - val_acc: 0.6223
Epoch 10/10
 - 768s - loss: 0.4709 - acc: 0.7788 - val_loss: 0.7472 - val_acc: 0.6292
0.5453205600589536


In [22]:
def lstm_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len):
    model = Sequential()
    e = Embedding(max_words, word_vec_size, weights=[embedding_matrix], input_length=max_sent_len, trainable=True)
    model.add(e)
    model.add(LSTM(100))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=32, verbose=2)
    y_pred = [int(round(i[0])) for i in model.predict(x_test)]
    limit_mem()
    return f1_score(y_test, y_pred)

In [23]:
print(lstm_pretrained_improved(x_train,x_test,y_train,y_test,embedding_matrix,max_words, word_vec_size,max_sent_len))

Train on 3327 samples, validate on 3328 samples
Epoch 1/10
 - 872s - loss: 0.6789 - acc: 0.5693 - val_loss: 0.6614 - val_acc: 0.6109
Epoch 2/10
 - 920s - loss: 0.5500 - acc: 0.7235 - val_loss: 0.6453 - val_acc: 0.6229
Epoch 3/10
 - 782s - loss: 0.3657 - acc: 0.8389 - val_loss: 0.7452 - val_acc: 0.6376
Epoch 4/10
 - 781s - loss: 0.2245 - acc: 0.9098 - val_loss: 0.9560 - val_acc: 0.6484
Epoch 5/10
 - 778s - loss: 0.1464 - acc: 0.9390 - val_loss: 1.2370 - val_acc: 0.6433
Epoch 6/10
 - 777s - loss: 0.1139 - acc: 0.9570 - val_loss: 1.1862 - val_acc: 0.6385
Epoch 7/10
 - 777s - loss: 0.0766 - acc: 0.9693 - val_loss: 1.6029 - val_acc: 0.6382
Epoch 8/10
 - 777s - loss: 0.0535 - acc: 0.9775 - val_loss: 1.6267 - val_acc: 0.6421
Epoch 9/10
 - 778s - loss: 0.0476 - acc: 0.9781 - val_loss: 1.5801 - val_acc: 0.6445
Epoch 10/10
 - 776s - loss: 0.0419 - acc: 0.9811 - val_loss: 1.8034 - val_acc: 0.6466
0.6066889632107023
